# Pandas Applying Function

In [1]:
# Importing Libraries
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt  

# Loading Data
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# Data Cleanup
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])

## Apply

In [2]:
help(df.apply)

Help on method apply in module pandas.core.frame:

apply(func: 'AggFuncType', axis: 'Axis' = 0, raw: 'bool' = False, result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None, args=(), by_row: "Literal[False, 'compat']" = 'compat', engine: "Literal['python', 'numba']" = 'python', engine_kwargs: 'dict[str, bool] | None' = None, **kwargs) method of pandas.core.frame.DataFrame instance
    Apply a function along an axis of the DataFrame.

    Objects passed to the function are Series objects whose index is
    either the DataFrame's index (``axis=0``) or the DataFrame's columns
    (``axis=1``). By default (``result_type=None``), the final return type
    is inferred from the return type of the applied function. Otherwise,
    it depends on the `result_type` argument.

    Parameters
    ----------
    func : function
        Function to apply to each column or row.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis along which the function is applied:

        

### Notes
- apply(): Apply functions to columns or rows.
- apply() cannot be used with a None values (null/NaN)

### Example 1
Calculate projected salaries next year, using an assumed rate of 3.0% for all roles.

In [3]:
df[pd.notna(df['salary_year_avg'])]['salary_year_avg']

28        109500.0
77        140000.0
92        120000.0
100       228222.0
109        89000.0
            ...   
785624    139216.0
785641    150000.0
785648    221875.0
785682    157500.0
785692    157500.0
Name: salary_year_avg, Length: 22003, dtype: float64

By using df.apply(), we must use copy()

In [4]:
df_salary = df[pd.notna(df['salary_year_avg'])].copy()

def projected_salary1(salary):
    return salary * 1.03

df_salary['salary_year_inflated'] = df_salary['salary_year_avg'].apply(projected_salary1)

df_salary[['salary_year_avg', 'salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


### Using df.apply() with lambda function

In [5]:
df_salary['salary_year_inflated'] = df_salary['salary_year_avg'].apply(lambda salary: salary * 1.03)

df_salary[['salary_year_avg', 'salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


### Example 3

#### Convert string into a list (job_skills column)

In [6]:
import ast

ast.literal_eval(df['job_skills'][1])

['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']

#### Regular function

In [7]:
def list_convert(skill_list):
    if pd.notna(skill_list):
        return ast.literal_eval(skill_list)

df['job_skills'] = df['job_skills'].apply(list_convert)

In [8]:
type(df['job_skills'][1])

list

#### Lambda function

In [9]:
df['job_skills'] = df['job_skills'].apply(lambda skill_lists: ast.literal_eval(skill_lists) if pd.notna(skill_lists) else skill_lists)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [8]:
type(df['job_skills'][1])

list

### Example 4
#### Calculate projected salaries next year, but:
- For senior roles (e.g., Senior Data Analysts), assume the rate is 5%
- For all other roles, assume rate is 3%

In [5]:
def projected_salary(row):
    if "Senior" in row['job_title_short']:
        return row['salary_year_avg'] * 1.05
    else:
        return row['salary_year_avg'] * 1.03
    
df_salary['salary_inflated'] = df_salary.apply(projected_salary, axis=1)

df_salary[['job_title_short', 'salary_year_avg', 'salary_inflated']]

,job_title_short,salary_year_avg,salary_inflated
28,Data Scientist,109500.0,NaN
77,Data Engineer,140000.0,NaN
92,Data Engineer,120000.0,NaN
100,Data Scientist,228222.0,NaN
109,Data Analyst,89000.0,NaN
...,...,...,...
785624,Data Engineer,139216.0,NaN
785641,Data Engineer,150000.0,NaN
785648,Data Scientist,221875.0,NaN
785682,Data Scientist,157500.0,NaN


#### Lambda function

In [8]:
df_salary['salary_inflated'] = df_salary.apply(lambda row: row['salary_year_avg']*1.05 if 'Senior' in row['job_title_short'] else row['salary_year_avg']*1.03, axis=1)

df_salary[['job_title_short', 'salary_year_avg', 'salary_inflated']]

,job_title_short,salary_year_avg,salary_inflated
28,Data Scientist,109500.0,112785.00
77,Data Engineer,140000.0,144200.00
92,Data Engineer,120000.0,123600.00
100,Data Scientist,228222.0,235068.66
109,Data Analyst,89000.0,91670.00
...,...,...,...
785624,Data Engineer,139216.0,143392.48
785641,Data Engineer,150000.0,154500.00
785648,Data Scientist,221875.0,228531.25
785682,Data Scientist,157500.0,162225.00
